In [1]:
import pandas as pd
df = pd.read_excel('DEMO15_최종(추추가.xlsx')

In [2]:
import warnings
warnings.filterwarnings('ignore')

from lightgbm import LGBMClassifier
import random
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
#맷플랏립 그래프 주피터에서 바로 출력하기
import seaborn as sns

from scipy import stats
from scipy.stats import t, norm, chi2, chi2_contingency
import re
import time

from matplotlib import rc
rc('font', family='Malgun Gothic')      #한글 폰트설정
plt.rcParams['axes.unicode_minus']=False      #마이너스 부호 출력 설정

from sklearn.datasets import load_iris
from sklearn import datasets

import cx_Oracle

from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Binarizer

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_auc_score

from sklearn import tree
from sklearn.datasets import make_classification

from sklearn.datasets import load_breast_cancer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

import xgboost    #1.7.5버전
from xgboost import plot_importance
from xgboost import XGBClassifier
from sklearn.datasets import load_breast_cancer

from hyperopt import hp
from hyperopt import fmin, tpe, Trials
from hyperopt import STATUS_OK

In [3]:
df.head()

,고객번호,성별,연령대,거주지역,멤버십,고객유형,경쟁사횟수,TARGET_15,TARGET_26,TARGET_37,...,A사비율48,B사비율48,C사비율48,D사비율48,구매최근15,구매최근26,구매최근37,구매최근6,구매최근7,구매최근8
0,1,M,노년층,서울,X,충성고객,15.0,1,0,0,...,0.760709,0.088626,0.150665,0.000000,3,4,9,4,9,2
1,2,M,노년층,서울,X,충성고객,8.0,1,1,1,...,0.945141,0.023511,0.000000,0.031348,1,7,2,7,2,2
2,3,M,노년층,서울,X,충성고객,NaN,1,0,0,...,0.000000,0.000000,1.000000,0.000000,1,2,9,2,9,4
3,4,F,노년층,서울,X,충성고객,3.0,1,0,0,...,0.676522,0.156522,0.166957,0.000000,2,1,1,1,1,1
4,5,M,노년층,서울,X,비충성고객,NaN,0,0,0,...,0.969828,0.000000,0.000000,0.030172,3,41,133,41,0,4


In [4]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib


# 한글 폰트 설정
font_path = 'C:/Windows/Fonts/malgun.ttf'  # 사용하고자 하는 한글 폰트 경로 지정
font_prop = fm.FontProperties(fname=font_path)
matplotlib.rc('font', family=font_prop.get_name())

In [5]:
# 널값 처리
df = df.dropna(subset=['고객번호'])
df['거주지역'] = df['거주지역'].fillna('-')
df = df.fillna(0)

In [7]:
df_encoded = pd.get_dummies(df[['연령대']])
df = pd.concat([df, df_encoded], axis=1)

In [6]:

# 레이블 인코딩
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

df['성별'] = label_encoder.fit_transform(df['성별'])
df['거주지역'] = label_encoder.fit_transform(df['거주지역'])
df['멤버십'] = label_encoder.fit_transform(df['멤버십'])
df['고객유형'] = label_encoder.fit_transform(df['고객유형'])
df.head(3)

,고객번호,성별,연령대,거주지역,멤버십,고객유형,경쟁사횟수,TARGET_15,TARGET_26,TARGET_37,...,A사비율48,B사비율48,C사비율48,D사비율48,구매최근15,구매최근26,구매최근37,구매최근6,구매최근7,구매최근8
0,1,1,노년층,8,1,1,15.0,1,0,0,...,0.760709,0.088626,0.150665,0.000000,3,4,9,4,9,2
1,2,1,노년층,8,1,1,8.0,1,1,1,...,0.945141,0.023511,0.000000,0.031348,1,7,2,7,2,2
2,3,1,노년층,8,1,1,0.0,1,0,0,...,0.000000,0.000000,1.000000,0.000000,1,2,9,2,9,4


In [9]:
df.head()

,고객번호,성별,연령대,거주지역,멤버십,고객유형,경쟁사횟수,TARGET_15,TARGET_26,TARGET_37,...,구매최근26,구매최근37,구매최근6,구매최근7,구매최근8,연령대_노년층,연령대_장년층,연령대_중년층,연령대_청년층,연령대_학생층
0,1,1,노년층,8,1,1,15.0,1,0,0,...,4,9,4,9,2,1,0,0,0,0
1,2,1,노년층,8,1,1,8.0,1,1,1,...,7,2,7,2,2,1,0,0,0,0
2,3,1,노년층,8,1,1,0.0,1,0,0,...,2,9,2,9,4,1,0,0,0,0
3,4,0,노년층,8,1,1,3.0,1,0,0,...,1,1,1,1,1,1,0,0,0,0
4,5,1,노년층,8,1,0,0.0,0,0,0,...,41,133,41,0,4,1,0,0,0,0


In [11]:
# 왜곡 정도가 심한 것 중 피처중요도가 높은 것 -> 로그변환
from scipy.stats import skew
column =['경쟁사횟수','계절총액_15','월평균액_6','월평균액_7','월평균액_8','계절총액_26','계절총액_37','계절총액_48','월평균액_15','월평균액_26','평균구매품목수6','평균구매품목수7','평균구매품목수8', '월평균액_37','평균구매품목수15','평균구매품목수26','평균구매품목수37','총방문횟수15','총방문횟수26','총방문횟수37','총방문횟수48']
skew_list = df[column].apply(lambda x: skew(x))
skew_list.sort_values(ascending=False)

월평균액_8       32.666507
월평균액_6       27.247061
월평균액_7       26.492022
계절총액_48      24.915955
월평균액_15      20.711374
계절총액_15      20.711374
월평균액_37      20.365979
계절총액_37      20.365979
월평균액_26      19.268723
계절총액_26      19.268723
경쟁사횟수         3.846571
평균구매품목수15     1.700088
평균구매품목수8      1.563800
평균구매품목수6      1.426665
평균구매품목수7      1.426027
평균구매품목수37     1.249097
평균구매품목수26     1.248002
총방문횟수48       0.645341
총방문횟수37       0.577547
총방문횟수26       0.531417
총방문횟수15       0.514767
dtype: float64

In [12]:
df2 = df[['TARGET_15','TARGET_26','고객유형','멤버십','계절총액_15','성별', 
          '등급_15', '월평균액_15', '평균구매품목수15', '총방문횟수15', '주말비율15', '심야비율_15', '오전비율_15', '오후비율_15', '저녁비율_15', '새벽비율_15', 'C0_누적_15', 'C0_절대_15', 'C1_누적_15', 'C1_절대_15', 'C2_누적_15', 'C2_절대_15', 'C3_누적_15', 'C3_절대_15', 'C4_누적_15',
          'C4_절대_15', 'C5_누적_15', 'C5_절대_15', 'C6_누적_15', 'C6_절대_15', 'C7_누적_15', 'C7_절대_15', 'C8_누적_15', 'C8_절대_15', 'C9_누적_15', 'C9_절대_15', 'C10_누적_15', 
          'C10_절대_15', 'C11_누적_15', 'C11_절대_15', 'C12_누적_15', 'C12_절대_15', 'C13_누적_15', 'C13_절대_15', 'A사비율15', 'B사비율15', 'C사비율15', 'D사비율15', 'T_누적_15',
          'T_절대_15' ,'구매최근15','연령대_노년층','연령대_장년층','연령대_중년층','연령대_청년층','연령대_학생층' ]].copy()

In [13]:
df2.head()

,TARGET_15,TARGET_26,고객유형,멤버십,계절총액_15,성별,등급_15,월평균액_15,평균구매품목수15,총방문횟수15,...,C사비율15,D사비율15,T_누적_15,T_절대_15,구매최근15,연령대_노년층,연령대_장년층,연령대_중년층,연령대_청년층,연령대_학생층
0,1,0,1,1,6.309726e+07,1,5,1.261945e+07,1.57,489.0,...,0.092689,0.000000,0,2,3,1,0,0,0,0
1,1,1,1,1,6.526469e+07,1,5,1.305294e+07,1.68,450.0,...,0.000000,0.000000,0,6,1,1,0,0,0,0
2,1,0,1,1,2.185141e+06,1,1,4.370282e+05,2.48,333.0,...,0.996368,0.000000,0,2,1,1,0,0,0,0
3,1,0,1,1,1.109015e+07,0,3,2.218029e+06,1.83,308.0,...,0.161348,0.000000,-1,5,2,1,0,0,0,0
4,0,0,0,1,9.912117e+06,1,3,1.982423e+06,1.69,356.0,...,0.000000,0.003333,1,11,3,1,0,0,0,0


In [14]:
df3 = df[['TARGET_26']].copy()

In [15]:
df4 = df[['TARGET_26','TARGET_37','고객유형','멤버십','성별', '계절총액_26', '등급_6','월평균액_6','평균구매품목수6','총방문횟수26','주말비율6',
        '심야비율_6', '오전비율_6', '오후비율_6', '저녁비율_6', '새벽비율_6', 
         'C0_누적_26', 'C0_절대_26', 'C1_누적_26', 'C1_절대_26', 'C2_누적_26', 'C2_절대_26', 
         'C3_누적_26', 'C3_절대_26', 'C4_누적_26', 'C4_절대_26', 'C5_누적_26', 'C5_절대_26',
         'C6_누적_26','C6_절대_26','C7_누적_26', 'C7_절대_26', 'C8_누적_26', 'C8_절대_26', 'C9_누적_26', 'C9_절대_26', 'C10_누적_26', 'C10_절대_26', 'C11_누적_26', 'C11_절대_26', 'C12_누적_26', 'C12_절대_26', 'C13_누적_26', 'C13_절대_26', 
          'A사비율26', 'B사비율26', 'C사비율26', 'D사비율26', 'T_누적_26', 'T_절대_26','구매최근6','연령대_노년층','연령대_장년층','연령대_중년층','연령대_청년층','연령대_학생층']].copy()

In [16]:
df4.head()

,TARGET_26,TARGET_37,고객유형,멤버십,성별,계절총액_26,등급_6,월평균액_6,평균구매품목수6,총방문횟수26,...,C사비율26,D사비율26,T_누적_26,T_절대_26,구매최근6,연령대_노년층,연령대_장년층,연령대_중년층,연령대_청년층,연령대_학생층
0,0,0,1,1,1,6.250124e+07,5,8.645718e+06,1.89,486.0,...,0.133758,0.000000,0,2,4,1,0,0,0,0
1,1,1,1,1,1,6.035115e+07,5,9.364654e+06,1.63,418.0,...,0.000000,0.021834,0,6,7,1,0,0,0,0
2,0,0,1,1,1,2.193845e+06,1,3.148029e+05,2.33,334.0,...,0.996341,0.000000,1,1,2,1,0,0,0,0
3,0,0,1,1,0,1.169704e+07,3,1.930887e+06,2.07,331.0,...,0.174837,0.000000,2,2,1,1,0,0,0,0
4,0,0,0,1,1,7.547575e+06,1,8.152466e+05,1.50,262.0,...,0.000000,0.008969,4,14,41,1,0,0,0,0


In [17]:
# 1~5기 데이터
xtrain = df2.drop('TARGET_26',axis=1)
ytrain = df2['TARGET_26']

# 6기 데이터
xtest = df4.drop('TARGET_37',axis=1)
ytest = df4['TARGET_37']

In [18]:
lgbm_wrapper = LGBMClassifier(n_estimators=400, learning_rate=0.05)

evals = [(xtrain, ytrain), (xtest, ytest)]
lgbm_wrapper.fit(xtrain, ytrain, early_stopping_rounds=50, eval_metric="logloss", eval_set=evals, verbose=True)
preds = lgbm_wrapper.predict(xtest)
pred_proba = lgbm_wrapper.predict_proba(xtest)[:, 1]

[1]	training's binary_logloss: 0.643751	valid_1's binary_logloss: 0.659342
[2]	training's binary_logloss: 0.639208	valid_1's binary_logloss: 0.655686
[3]	training's binary_logloss: 0.63509	valid_1's binary_logloss: 0.652863
[4]	training's binary_logloss: 0.631231	valid_1's binary_logloss: 0.650585
[5]	training's binary_logloss: 0.627687	valid_1's binary_logloss: 0.64807
[6]	training's binary_logloss: 0.624448	valid_1's binary_logloss: 0.646129
[7]	training's binary_logloss: 0.62138	valid_1's binary_logloss: 0.643742
[8]	training's binary_logloss: 0.618456	valid_1's binary_logloss: 0.641763
[9]	training's binary_logloss: 0.615815	valid_1's binary_logloss: 0.640005
[10]	training's binary_logloss: 0.613277	valid_1's binary_logloss: 0.63825
[11]	training's binary_logloss: 0.610904	valid_1's binary_logloss: 0.636805
[12]	training's binary_logloss: 0.608644	valid_1's binary_logloss: 0.635681
[13]	training's binary_logloss: 0.606548	valid_1's binary_logloss: 0.634589
[14]	training's binary_lo

In [19]:
def get_clf_eval(ytest, pred=None, pred_proba=None):
    confusion = confusion_matrix( ytest, pred)
    accuracy = accuracy_score(ytest , pred)
    precision = precision_score(ytest , pred)
    recall = recall_score(ytest , pred)
    f1 = f1_score(ytest,pred)
    # ROC-AUC 추가 
    roc_auc = roc_auc_score(ytest, pred_proba)
    print('오차 행렬')
    print(confusion)
    # ROC-AUC print 추가
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

In [20]:
get_clf_eval(ytest, preds, pred_proba)

오차 행렬
[[ 1073  6209]
 [  546 11555]]
정확도: 0.6515, 정밀도: 0.6505, 재현율: 0.9549,    F1: 0.7738, AUC:0.6733


In [22]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

# Define the parameter grid
param_grid = {
    'penalty': [ 'l2'],
    'C': [0.01, 0.1, 1.0, 10.0],
    'solver': ['liblinear', 'saga'],
    'max_iter': [100, 200, 300]
}

# Create the logistic regression model
logreg = LogisticRegression()

# Create the scaler for feature scaling
scaler = StandardScaler()

# Scale the training data
X_train_scaled = scaler.fit_transform(xtrain)

# Create the grid search object
grid_search = GridSearchCV(logreg, param_grid, cv=5, n_jobs = -1)

# Perform grid search to find the best model
grid_search.fit(X_train_scaled, ytrain)

# Get the best model and its hyperparameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Print the best hyperparameters
print("Best Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")


Best Hyperparameters:
C: 1.0
max_iter: 100
penalty: l2
solver: liblinear


In [27]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# 표준화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(xtrain)
X_validation_scaled = scaler.transform(xtest)


# 로지스틱 회귀 모델 생성 및 학습
logreg = LogisticRegression(penalty='l2', C=1.0, solver='liblinear', max_iter=100)
logreg.fit(X_train_scaled, ytrain)

# 검증 데이터셋으로 모델 평가
validation_accuracy = logreg.score(X_validation_scaled, ytest)



# 검증 데이터셋 예측 확률 계산
validation_probabilities = logreg.predict_proba(X_validation_scaled)
print("Validation Predicted Probabilities:")
print(validation_probabilities)

# 검증 데이터셋 정확도 계산
validation_accuracy = logreg.score(X_validation_scaled, ytest)
print("Validation Accuracy:", validation_accuracy)


Validation Predicted Probabilities:
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]
Validation Accuracy: 0.3768766444822783
